Houseplant Soil Moisture Model
Version 0.0 Percolation and Evaporation - No Plant
Use this version for calibration

Physical processes:
1. Surface evaporation - loss due to air
2. Gravitational percolation - downward drainage

Model: 1D vertical soil column with 1cm3 cells

In [5]:
import sys
!{sys.executable} -m pip install numpy

Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
  Using cached https://www.piwheels.org/simple/numpy/numpy-2.0.2-cp39-cp39-linux_armv7l.whl (5.8 MB)


In [6]:
import sys
!{sys.executable} -m pip install datetime

Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
  Using cached https://www.piwheels.org/simple/datetime/DateTime-5.5-py3-none-any.whl (52 kB)
  Using cached https://www.piwheels.org/simple/zope-interface/zope_interface-8.0.1-cp39-cp39-linux_armv7l.whl (245 kB)
  DEPRECATION: Wheel filename 'pytz-2013d-py3-none-any.whl' is not correctly normalised. Future versions of pip will raise the following error:
  Invalid wheel filename (invalid version): 'pytz-2013d-py3-none-any'
  
   pip 25.3 will enforce this behaviour change. A possible replacement is to rename the wheel to use a correctly normalised name (this may require updating the version in the project metadata). Discussion can be found at https://github.com/pypa/pip/issues/12938
  DEPRECATION: Wheel filename 'pytz-2012j-py3-none-any.whl' is not correctly normalised. Future versions of pip will raise the following error:
  Invalid wheel filename (invalid version): 'pytz-2012j-py3-none-any'
  
   pip 25.3 wi

In [8]:
import sys
!{sys.executable} -m pip install matplotlib --no-deps

Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
  Using cached https://www.piwheels.org/simple/matplotlib/matplotlib-3.9.4-cp39-cp39-linux_armv7l.whl (7.9 MB)


In [8]:
#These are basic libraries we need.
import numpy as np #math
from datetime import datetime #calendar

Parameters - Adjust these parameters to match the experimental setup
1. Model geometry
   a. Soil depth (cm), number of cells
   b. Cell size (cm3), unit cell
   c. sensor depth (cm), measurement point
2. Soil properties
   a. Maximum water content, i.e. porosity, i.e. saturation percentage (%)
   b. Water held against gravity (%)
   c. Wilting point for future use (%) - for future use
   d. Initial moisture (%) - moisture after watering
3. Process parameters to fit our sensor data
   a. Surface evaporation rate (1/hour)
   b. Percolation coefficient (dimensionless)
4. Simulation settings
   a. Time steps (hours) - number of hours in simulation 168 =  one week
   b. Time step (hous) - unit time step

In [2]:
#initialize parameters

#soil column
DEPTH = 20.0 #total soil depth (cm)
CELL_SIZE = 1.0 #Cell size (cm) 1cm3 volume
SENSOR_DEPTH = 7.6 #Capacitive sensor depth (cm) - 3 inches

#soil properties (typical for potting mix)
SATURATION = 0.6 #Maximum water content (porosity)
FIELD_CAPACITY = 0.35 #Water held against gravity
WILTING_POINT = 0.12 #Plant wilting threshold (for future use)
INITIAL_MOISTURE = 0.35 #Moisture after watering

#process parameters
EVAP_RATE = 0.002 #surface evaporation rate (1/hour)
PERCOLATION_COEFF = 0.08 #drainage coefficient (dimensionless)

#simulation settings
TIME_STEPS = 168 #total simulation time (hours) - 1 week
DT = 1.0 #time step (hours)


In [4]:
#initialize model

print("="*60)
print("SOIL MOISTURE SIMULATION")
print("="*60)

#create soil column grid
nz = int(DEPTH/ CELL_SIZE)
sensor_cell = int(SENSOR_DEPTH / CELL_SIZE)

print(f"Soil column: {nz} cells ({DEPTH} cm depth")
print(f"Cell volume: {CELL_SIZE}3 = 1cm3")
print(f"Sensor position: Cell {sensor_cell} ({SENSOR_DEPTH} cm depth)")
print(f"Simulation time: {TIME_STEPS} hours ({TIME_STEPS/24:.1f} days)")
print(f"Time step: {DT} hour(s)")
print()

#Initialize moisture array
moisture = np.full(nz, INITIAL_MOISTURE)

#Storage for results
time_array = []
surface_moisture = []
sensor_moisture = []
average_moisture = []
bottom_moisture = []

SOIL MOISTURE SIMULATION
Soil column: 20 cells (20.0 cm depth
Cell volume: 1.03 = 1cm3
Sensor position: Cell 7 (7.6 cm depth)
Simulation time: 168 hours (7.0 days)
Time step: 1.0 hour(s)



In [10]:
#run simulation

print("Running simulation...")
start_time = datetime.now()

for t in range(TIME_STEPS):
    #create copy for updating
    new_moisture = moisture.copy()

    #1. SURFACE EVAPORATION
    # evaporation slows as surface dries (nonlinear)

    surface_dryness = moisture[0] / FIELD_CAPACITY
    effective_evap = EVAP_RATE * min(1.0, surface_dryness)*DT
    new_moisture[0] = max(0.0, moisture[0] - effective_evap)

    #2 PERCOLATION (gravity drainage)
    # water only moves downward when moisture exceeds field capacity

    for z in range(nz-1):
        excess_water = max(0.0, moisture[z] - FIELD_CAPACITY)
        if excess_water > 0:
            #flow rate proportional to excess water
            flow = PERCOLATION_COEFF * excess_water * DT
            new_moisture[z] -= flow
            new_moisture[z+1] += flow

            #Prevent exceeding saturation (overflow cascades down)
            if new_moisture[z+1] > SATURATION:
                overflow = new_moisture[z+1] - SATURATION
                new_moisture[z+1] = SATURATION
                if z+2 < nz:
                    new_moisture[z+2] += overflow
    
    #update moisture array
    moisture = new_moisture

    #3 RECORD DATA
    time_array.append(t * DT)
    surface_moisture.append(moisture[0])
    sensor_moisture.append(moisture[sensor_cell])
    average_moisture.append(np.mean(moisture))
    bottom_moisture.append(moisture[-1])

    elapsed = (datetime.now() - start_time).total_seconds()
    print(f"Simulation complete in {elapsed:.3f} seconds")
    print()

#ANALYSIS

print("RESULTS:")
print(f"Initial sensor reading: {INITIAL_MOISTURE:.3f}")
print(f"Final sensor reading: {sensor_moisture[-1]:.3f}")
print(f"Moisture decrease: {(INITIAL_MOISTURE - sensor_moisture[-1]):.3f}")

#find time to 50% dryness
target_moisture = INITIAL_MOISTURE * 0.5
try:
    idx = next (i for i, m in enumerate(sensor_moisture) if m < target_moisture)
    print(f" Time to 50% dryness: {time_aray[idx]:.1f} hours ({time_array[idx]/24:.1f} days)")
except StopIteration:
    print("Did not reach 50% dryness in simulation period")

print()

#VISUALIZATION
"""
fig, axes = plt.subplots(2, 1, figsize = (12,10))
#---Plot 1: Moisture vs. time---
ax1 = axes[0]
ax1.plot(time_array, sensor_moisture, 'purple', linewidth = 3, label = f'Sensor at {SENSOR_DEPTH} cm (YOUR RASPBERRY PI)')
ax1.plot(time_array, surface_moisture, 'red', linewidth = 2, label = 'Surface (0 cm)')
ax1.plot(time_array, average_moisture, 'blue', linewidth = 2, linestyle = "--", label = 'Column Average')
#Add field capacity reference line
ax1.axhline(y=FIELD_CAPACITY, color = 'orange', linestyle = ":", label = 'Field Capacity')
ax1.axhline(y= WILTING_POINT, color = 'brown', linestyle = ":", label = 'Wilting Point')
ax1.set_xlabel('Time (hours)', fontsize = 12)
ax1.set_ylabel('Volumetric Moisture Content', fontsize = 12)
ax1.set_title('Soil Moisture Dynamics - Compare to Your Sensor Data', fontsize = 14, fontweight = 'bold')
ax1.legend(loc = 'best')
ax1.grid(True, alpha = 0.3)
ax1.set_ylim([0, SATURATION])

#---Plot 2: Depth profile evolution---
ax2=axes[1]
depths = np.arrange(nz) * CELL_SIZE
#Plot profiles at different times
snapshot_times = [0, 24, 72, 168] #hours
for t_snap in snapshot_times:
    if t_snap < TIME_STEPS:
        idx = int(t_snap / DT)
        #Reconstruct moisture profile (approximation)
        #For proper depth profiles, we'd need to store full arrays
        label f't = {t_snap}h'
        if t_snap == 0:
            profile = no.full(nz, INITIAL_MOISTURE)
        else:
            #This is illustrative - in reality you'd store full profiles
            profile = np.linspace(surface_moisture[idx], bottom_moisture[idx], nz)
            ax2.plot(profile, depths, linewidth=2, label=label)
            #Mark sensor depth
            ax2.axhline(y=SENSOR_DEPTH, color='purple', linestyle = "--", linewidth = 2, label = 'Sensor Depth')
            x2.set_xlabel('Moisture Content', fontsize = 12)
            ax2.set_ylabel('Depth (cm)', fontsize = 12)
            ax2.set_title('Moisture Profile vs Depth', fontsize = 14, fontweight = 'bold')
            ax2.legend(loc='best')
            ax2.grid(True, alpha = 0.3)
            ax2.invert_yaxis() #Depth increases downward

            plt.tight_layouy()
            plt.savefig('soil_moisture_results.png, dpi = 150, bbox_inches = 'tight')
            print("Plot saved as soil_moisture_results.png")
            plt.show()
"""
#EXPORT DATA FOR COMPARISON WITH EXPERIMENTAL RESULTS

#Create data array
output_data = np.column_stack([
    time_array,
    sensor_moisture,
    surface_moisture,
    average_moisture
])

#Save to .csv
np.savetxt('soil_moisture_simulation.csv',output_data, delimiter = ',', header= 'time_hours,sensor_moisture,surface_moisture,average_moisture',comments ='')
print("Data exported to 'soil_moisture_simulation.csv'")
print()

#CALIBRATION INSTRUCTIONS
print("="*60)
print("CALIBRATION GUIDE FOR MAKERS CLUB")
print("="*60)
print("""
1. COLLECT EXPERIMENTAL DATA
    -Water soil until saturated.
    -Record Raspberry Pi sensor reading immediately
    -Log sensor values every 2-4 hours for 3-7 days
    -Save as csv with columns: time_hours, sensor_reading

2. TUNE PARAMETERS
    -INITIAL_MOISTURE: Set to your first sensor reading
    -EVAP_RATE: Adjust to match surface drying speed (Higher = faster evaporation, typical range: 0.001-0.005)
    -PERCOLATION_COEFF: Adjust to match drainage speed
        (Higher = faster drainage, typical range: 0.05-0.15)

3. COMPARE
    -Plot your real sensor over the purple curve
    -Iterate parameters until curves match

4. NEXT STEPS:
    -Add plant water update (root absorption)
    -Compare planted pot vs control pot
    -Add environmental factors (temp, humidity)
""")

    

Running simulation...
Simulation complete in 0.002 seconds

Simulation complete in 0.002 seconds

Simulation complete in 0.002 seconds

Simulation complete in 0.003 seconds

Simulation complete in 0.003 seconds

Simulation complete in 0.003 seconds

Simulation complete in 0.004 seconds

Simulation complete in 0.004 seconds

Simulation complete in 0.004 seconds

Simulation complete in 0.004 seconds

Simulation complete in 0.005 seconds

Simulation complete in 0.005 seconds

Simulation complete in 0.005 seconds

Simulation complete in 0.006 seconds

Simulation complete in 0.006 seconds

Simulation complete in 0.006 seconds

Simulation complete in 0.007 seconds

Simulation complete in 0.007 seconds

Simulation complete in 0.007 seconds

Simulation complete in 0.008 seconds

Simulation complete in 0.008 seconds

Simulation complete in 0.008 seconds

Simulation complete in 0.008 seconds

Simulation complete in 0.009 seconds

Simulation complete in 0.010 seconds

Simulation complete in 0.010

In [ ]:
T